    Lucas Tsunaki
    02.07.2024

In this example we simulate ambiguous resonances in multipulse quantum sensing with nitrogen vacancy (NV) centers in diamonds. This notebook closely follows the work from "Ambiguous Resonances in Multipulse Quantum Sensing with Nitrogen Vacancy Centers" available at . 

As this is a quite complex problem, physics discussions will be kept to a minimum here in favor of a focus to the simulation. For a detailed explanation of the problem, please refer to the original paper.

**Table of contents**<a id='toc0_'></a>    
- 1. [Field Misalignment and $^{15}\text{N}$ Coupling](#toc1_)    
- 2. [RXY8 with H2](#toc2_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [ ]:
import numpy as np

# 1. <a id='toc1_'></a>[Field Misalignment and $^{15}\text{N}$ Coupling](#toc0_)

We begin simulating a XY8-2 sequence with a misaligned magnetic field, as in section IV of the paper.

In [ ]:
from quaccatoo.QSys import NV

# define the NV center system
qsys = NV(
    N=15, # here we start considering the 15N nuclear spin
    B0 = 39, # external magnetic field in mT
    units_B0='mT',
    theta=3.8, # 3.8 degrees os misalignment
    units_angles='deg'
)

w1 = 40

from quaccatoo.PredefDDSeqs import XY8

XY8_15N  = XY8(
    M=2,
    free_duration = np.linspace(.25, .36, 100), # free evolution time in us
    pi_pulse_duration = 1/2/w1, # pi-pulse duration in us
    system = qsys, # NV center system
    H1 = w1*qsys.MW_H1, # control Hamiltonian
    pulse_params = {'f_pulse': qsys.MW_freqs[1]}, # MW frequency for the ms=0 --> ms=+1 state transition
    time_steps=200 # Number of time steps in each MW pulse
)

# plot the pulses
XY8_15N.plot_pulses(tau=.1, figsize=(12, 4))

Here we can observe the 16 $\pi$-pulses plus the initial and final $\pi/2$ pulses. Now to run the experiment.

In [ ]:
from quaccatoo.Analysis import Analysis
XY8_15N.run()
Analysis(XY8_15N).plot_results(title=r'XY8-2 $B_0=39$ mT $\theta_0=3.8^\circ$')

Increasing the XY8 order for M=4,6,8:

In [ ]:
# M = 4
XY8_15N.M = 4
XY8_15N.run()
Analysis(XY8_15N).plot_results(title=r'XY8-4 $B_0=39$ mT $\theta_0=3.8^\circ$')

In [ ]:
# M = 6
XY8_15N.M = 6
XY8_15N.run()
Analysis(XY8_15N).plot_results(title=r'XY8-6 $B_0=39$ mT $\theta_0=3.8^\circ$')

In [ ]:
# M = 8
XY8_15N.M = 8
XY8_15N.run()
Analysis(XY8_15N).plot_results(title=r'XY8-8 $B_0=39$ mT $\theta_0=3.8^\circ$')

Now, changing the field for XY8-2.

In [ ]:
# 33 mT, 5.1 deg
qsys = NV(
    N=15, # here we start considering the 15N nuclear spin
    B0 = 33, # external magnetic field in mT
    units_B0='mT',
    theta=5.1, # 2 degrees os misalignment
    units_angles='deg'
)

XY8_15N  = XY8(
    M=2,
    free_duration = np.linspace(.22, .45, 100), # free evolution time in us
    pi_pulse_duration = 1/2/w1, # pi-pulse duration in us
    system = qsys, # NV center system
    H1 = w1*qsys.MW_H1, # control Hamiltonian
    pulse_params = {'f_pulse': qsys.MW_freqs[1]} # MW frequency for the ms=0 --> ms=+1 state transition
)

XY8_15N.run()
Analysis(XY8_15N).plot_results(title=r'XY8-2 $B_0=33$ mT $\theta_0=5.1^\circ$')

In [ ]:
# 26 mT, 9.3 deg
qsys = NV(
    N=15,
    B0 = 26, 
    units_B0='mT',
    theta = 9.3,
    units_angles='deg'
)

XY8_15N  = XY8(
    M=2,
    free_duration = np.linspace(.22, .45, 100), # free evolution time in us
    pi_pulse_duration = 1/2/w1, # pi-pulse duration in us
    system = qsys, # NV center system
    H1 = w1*qsys.MW_H1, # control Hamiltonian
    pulse_params = {'f_pulse': qsys.MW_freqs[1]} # MW frequency for the ms=0 --> ms=+1 state transition
)

XY8_15N.run()
Analysis(XY8_15N).plot_results(title=r'XY8-2 $B_0=26$ mT $\theta_0=9.3^\circ$')

In [ ]:
# 17 mT, 17 deg
qsys = NV(
    N=15, # here we start considering the 15N nuclear spin
    B0 = 17, # external magnetic field in mT
    units_B0='mT',
    theta=17, # 2 degrees os misalignment
    units_angles='deg'
)

XY8_15N  = XY8(
    M=2,
    free_duration = np.linspace(.25, .65, 100), # free evolution time in us
    pi_pulse_duration = 1/2/w1, # pi-pulse duration in us
    system = qsys, # NV center system
    H1 = w1*qsys.MW_H1, # control Hamiltonian
    pulse_params = {'f_pulse': qsys.MW_freqs[1]} # MW frequency for the ms=0 --> ms=+1 state transition
)

XY8_15N.run()
Analysis(XY8_15N).plot_results()

# 2. <a id='toc2_'></a>[RXY8 with H2](#toc0_)

In [ ]:
from qutip import tensor, jmat, qeye
# 17 mT, 17 deg
qsys = NV(
    N=15, # here we start considering the 15N nuclear spin
    B0 = 40, # external magnetic field in mT
    units_B0='mT',
)

w1 = 20
f2 = 5.5
B2z = 0.28

def B2(t, args):
    return B2z*np.sin(f2*t)

H2 = [tensor(jmat(1,'z'), qeye(2)), B2]      

XY8_12_H2  = XY8(
    M=12,
    free_duration = np.linspace(.06, 0.17, 100), # free evolution time in us
    pi_pulse_duration = 1/2/w1, # pi-pulse duration in us
    system = qsys, # NV center system
    H1 = w1*qsys.MW_H1, # control Hamiltonian
    pulse_params = {'f_pulse': qsys.MW_freqs[0]},
    H2 = H2
)
XY8_12_H2.run()
Analysis(XY8_12_H2).plot_results()